In [32]:
from subprocess import Popen, PIPE
import os
import pathlib
import shutil
import os
import pandas as pd
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

In [2]:
#rice
path_mites_dire = 'data/rice/by_family/'
path_out_dir = 'data/rice/by_cluster/'
pid = 0.97
what = 'rice'
min_count = 10
workers = 5

In [41]:
#wheat
path_mites_dire = 'data/wheat/by_family/'
path_out_dir = 'data/wheat/by_cluster/'
path_out_dir_clus = 'data/wheat/clusters_similar/'
pid = 0.97
path_out_fasta = 'data/wheat/similar.fasta'
path_similar = 'data/wheat/similar_mites_count.csv'
min_count = 10
workers = 5

In [ ]:
files = [f for f in os.listdir(path_mites_dire) if os.path.isfile(os.path.join(path_mites_dire, f))]
count = 0
total = len(files)
for file in files:
    file_path = path_mites_dire + file
    count += 1
    print("%i / %i" % (count,total))
    c_dir = path_out_dir + file + "/"
    if os.path.exists(c_dir):
        continue
    if os.path.isdir(c_dir):
        shutil.rmtree(c_dir)
    pathlib.Path(c_dir).mkdir(parents=True, exist_ok=True)
    cmd_list = [
    './bin/vsearch-2.11.1/bin/vsearch',
    '--cluster_fast', file_path,
    '--threads',str(workers),
    '--strand','both',
    '--clusters', c_dir + "c_",
    '--iddef','1',
    '--id', str(pid)]
    p = Popen(cmd_list, stdout=PIPE, stderr=PIPE)
    out,err = p.communicate()


In [ ]:
files = [f for f in os.listdir(path_mites_dire) if os.path.isfile(os.path.join(path_mites_dire, f))]
count = 0
total = len(files)
res = {}
for file in files:
    file_path = path_mites_dire + file
    c_dir = path_out_dir + file + "/"
    clusters = [f for f in os.listdir(c_dir) if os.path.isfile(os.path.join(c_dir, f))]
    seqs = []
    for cluster in clusters:
        file_c = open(c_dir + cluster, "r")
        count_seqs = 0
        for line in file_c:
            if line[0:1] == ">":
                count_seqs += 1
        seqs.append(count_seqs)
    if max(seqs) > min_count:
        res[file] = (len(clusters), max(seqs))
        #print(file + ": " + str(len(clusters)) + " max: " + str(max(seqs)))
        #print("*" * 10)
    count += 1
    print("%i / %i" % (count,total))

In [6]:
res_s = sorted(res.items(), key=lambda x:x[1][0])

In [12]:

with open(path_similar, 'w') as f:
    for v in res_s:
        f.write("%s,%s,%s\n"%(v[0],v[1][0],v[1][1]))


'rice'

In [ ]:
#extract fasta of top hits

In [23]:
df_similar = pd.read_csv(path_similar, sep=',', header=None)

In [24]:
df_similar.columns = ['MITE','clusters','counts']

In [26]:
df_similar = df_similar[df_similar.counts > 100]

In [28]:
pd.set_option('display.max_colwidth', -1)


In [ ]:
buffer_mites = []
for k,v in df_similar.iterrows():
    mite = v.MITE
    count = v.counts
    file_path = path_mites_dire + mite
    fasta_seq = SeqIO.parse(file_path, 'fasta')
    first_record = next(fasta_seq)
    buffer_mites.append(first_record)
    print(mite, count)

In [39]:
SeqIO.write(buffer_mites, path_out_fasta, "fasta")

1723

In [42]:
cmd_list = [
'./bin/vsearch-2.11.1/bin/vsearch',
'--cluster_fast', path_out_fasta,
'--threads',str(workers),
'--strand','both',
'--clusters', path_out_dir_clus + "c_",
'--iddef','1',
'--id', '0.6']
p = Popen(cmd_list, stdout=PIPE, stderr=PIPE)
out,err = p.communicate()
